In [9]:
DATA_PATH = 'C:\\Users\\tomte\\SGDatavers\\football_euro\\'
RANKING_PATH = f'{DATA_PATH}fifa_ranking-2023-07-20.csv'
MATCH_PATH = f'{DATA_PATH}international_matches.csv'
RESULT_PATH = f'{DATA_PATH}results.csv'

In [54]:
import numpy as np
import pandas as pd 

### 0) Read Data

In [10]:
ranking_data = pd.read_csv(RANKING_PATH)
match_data = pd.read_csv(MATCH_PATH)
result_data = pd.read_csv(RESULT_PATH)

In [16]:
ranking_data.head(3)

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,1,Germany,GER,57.0,0.0,0,UEFA,1992-12-31
1,96,Syria,SYR,11.0,0.0,0,AFC,1992-12-31
2,97,Burkina Faso,BFA,11.0,0.0,0,CAF,1992-12-31


In [17]:
match_data.head(3)

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
0,1993-08-08,Bolivia,Uruguay,South America,South America,59,22,0,0,3,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1993-08-08,Brazil,Mexico,South America,North America,8,14,0,0,1,...,No,Draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993-08-08,Ecuador,Venezuela,South America,South America,35,94,0,0,5,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
result_data.head(3)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False


### 1) Filter Data by Euro

In [33]:
ranking_uefa = ranking_data[ranking_data['confederation'] == 'UEFA']

In [66]:
european_countries: list = list(ranking_uefa['country_full'].unique())

In [77]:
home_team_mask = match_data['home_team_continent'] == 'Europe'
away_team_mask = match_data['away_team_continent'] == 'Europe'

In [78]:
team_masks = np.logical_and.reduce([home_team_mask, away_team_mask])

In [79]:
match_europe = match_data[team_masks] 

In [80]:
match_europe.head(3)

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
8,1993-08-11,Faroe Islands,Norway,Europe,Europe,111,9,0,0,0,...,No,Lose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1993-08-11,Sweden,Switzerland,Europe,Europe,4,3,0,0,1,...,No,Lose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,1993-08-22,Sweden,France,Europe,Europe,4,12,0,0,1,...,No,Draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
match_europe.shape

(6529, 25)

In [82]:
home_country_mask = result_data['home_team'].isin(european_countries)
away_country_mask = result_data['away_team'].isin(european_countries)

In [83]:
country_masks = np.logical_and.reduce([home_country_mask, away_country_mask])

In [84]:
result_europe = result_data[country_masks] 

In [85]:
result_europe

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
46136,2023-11-21,Croatia,Armenia,1,0,UEFA Euro qualification,Zagreb,Croatia,False
46137,2023-11-21,Andorra,Israel,0,2,UEFA Euro qualification,Andorra la Vella,Andorra,False
46138,2023-11-21,Kosovo,Belarus,0,1,UEFA Euro qualification,Pristina,Kosovo,False
46139,2023-11-21,Romania,Switzerland,1,0,UEFA Euro qualification,Bucharest,Romania,False
